In [1]:
#Se importan las librerías necesarias para el problema
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import missingno as msng
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
plt.rcParams['figure.figsize'] = (15, 10)

In [2]:
df = pd.read_csv('../data/df_procesado.csv')

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
import xgboost as xgb
from xgboost import plot_importance
from sklearn.metrics import roc_auc_score
from numpy import mean
from sklearn.ensemble import RandomForestClassifier
import wandb


In [4]:
data = shuffle(df[['OPERA', 'MES', 'periodo_dia','TIPOVUELO', 'SIGLADES', 'DIANOM', 'atraso_15']], random_state = 111)

In [5]:
features = pd.concat([pd.get_dummies(data['OPERA'], prefix = 'OPERA'),pd.get_dummies(data['TIPOVUELO'], prefix = 'TIPOVUELO'),pd.get_dummies(data['periodo_dia'], prefix = 'periodo_dia'), pd.get_dummies(data['MES'], prefix = 'MES')], axis = 1)
label = data['atraso_15']

In [6]:
features_importantes = features[['MES_7', 'TIPOVUELO_I', 'OPERA_Copa Air', 'OPERA_Latin American Wings',
                                'MES_12', 'OPERA_Grupo LATAM', 'MES_10', 'OPERA_JetSmart SPA', 'OPERA_Air Canada',
                                'MES_9', 'OPERA_American Airlines']]
labels_ = label

# PIPELINE DE MONITOREO
Para probar distintos modelos y comparar sus métricas de forma cómoda, es buena práctica usar librerías que permitan hacer seguimiento de los distintos modelos entrenados, a modo de poder comparar y seleccionar correctamente los modelos de forma compacta. En este caso hacemos uso de W&B

Dentro de la pipeline se incluyen procesos como Cross Validation, Oversampling, Undersampling.

In [7]:
#nuestro primer hiperparametro a testear es el under-over-sampling para la data desbalanceada.
sweep_configuration = {
    'method': 'grid',
    'name': 'sweep',
    'metric': {'goal': 'maximize', 'name': 'precision'},
    'parameters': 
    {
        'over_values': {'values': [0.3, 0.4, 0.5]},
        'under_values': {'values': [0.7, 0.6, 0.5]}
     }
}

#creamos un diccionario con modelos base clasicos para clasificacion que seran sometidos a test

models = {
    'SVM' : SVC(),
    'XGB' : xgb.XGBClassifier(random_state=1, learning_rate=0.01),
    'RF'  : RandomForestClassifier(random_state = 1),
    'LR'  : LogisticRegression()
 }
sweep_id = wandb.sweep(sweep=sweep_configuration, project='cw-challenge-model-test')

Create sweep with ID: aiz2vglq
Sweep URL: https://wandb.ai/ignacio-onate/cw-challenge-model-test/sweeps/aiz2vglq


In [8]:
import wandb

X = features
y = label

def train():
    wandb.init()
    for name, model in models.items():
        # define pipeline
        over = SMOTE(sampling_strategy=wandb.config.over_values)
        under = RandomUnderSampler(sampling_strategy=wandb.config.under_values)
        steps = [('over', over), ('under', under), ('model', model)]
        pipeline = Pipeline(steps=steps)
        # evaluate pipeline
        scores = cross_validate(pipeline, X, y, scoring=['accuracy','f1','precision','recall','roc_auc'], cv=5, n_jobs=-1)
        #score = mean(scores)
            
        wandb.log({
            'accuracy_'+name: mean(scores['test_accuracy']),
            'f1_'+name : mean(scores['test_f1']),
            'precision_'+name : mean(scores['test_precision']),
            'recall_'+name : mean(scores['test_recall']),
            'roc_auc_'+name : mean(scores['test_roc_auc'])
        })
        print("Modelo: ", name)
        #print(scores)
wandb.agent(sweep_id, function = train)

wandb: Agent Starting Run: 2pwh13jl with config:
wandb: 	over_values: 0.3
wandb: 	under_values: 0.7
wandb: Currently logged in as: ignacio-onate. Use `wandb login --relogin` to force relogin


Modelo:  SVM
Modelo:  XGB
Modelo:  RF
Modelo:  LR


accuracy_LR,▁
accuracy_RF,▁
accuracy_SVM,▁
accuracy_XGB,▁
f1_LR,▁
f1_RF,▁
f1_SVM,▁
f1_XGB,▁
precision_LR,▁
precision_RF,▁
precision_SVM,▁


wandb: Agent Starting Run: 1ssmyda8 with config:
wandb: 	over_values: 0.3
wandb: 	under_values: 0.6


Modelo:  SVM
Modelo:  XGB
Modelo:  RF
Modelo:  LR


...

accuracy_LR,▁
accuracy_RF,▁
accuracy_SVM,▁
accuracy_XGB,▁
f1_LR,▁
f1_RF,▁
f1_SVM,▁
f1_XGB,▁
precision_LR,▁
precision_RF,▁
precision_SVM,▁


wandb: Agent Starting Run: 7gmctfvl with config:
wandb: 	over_values: 0.5
wandb: 	under_values: 0.5


Modelo:  SVM
Modelo:  XGB
Modelo:  RF
Modelo:  LR


accuracy_LR,▁
accuracy_RF,▁
accuracy_SVM,▁
accuracy_XGB,▁
f1_LR,▁
f1_RF,▁
f1_SVM,▁
f1_XGB,▁
precision_LR,▁
precision_RF,▁
precision_SVM,▁


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


In [12]:
sweep_configuration2 = {
    'method': 'grid',
    'name': 'sweep',
    'metric': {'goal': 'maximize', 'name': 'precision'},
    'parameters': 
    {
        'over_values': {'values': [0.3, 0.4, 0.5]},
        'under_values': {'values': [0.7, 0.6, 0.5]}
     }
}
sweep_id2 = wandb.sweep(sweep=sweep_configuration2, project='cw-challenge-model-test-important-features')

Create sweep with ID: x56k4uit
Sweep URL: https://wandb.ai/ignacio-onate/cw-challenge-model-test-important-features/sweeps/x56k4uit


In [13]:
import wandb

X = features_importantes
y = labels_

def train():
    wandb.init()
    for name, model in models.items():
        # define pipeline
        over = SMOTE(sampling_strategy=wandb.config.over_values)
        under = RandomUnderSampler(sampling_strategy=wandb.config.under_values)
        steps = [('over', over), ('under', under), ('model', model)]
        pipeline = Pipeline(steps=steps)
        # evaluate pipeline
        scores = cross_validate(pipeline, X, y, scoring=['accuracy','f1','precision','recall','roc_auc'], cv=5, n_jobs=-1)
        #score = mean(scores)
            
        wandb.log({
            'accuracy_'+name: mean(scores['test_accuracy']),
            'f1_'+name : mean(scores['test_f1']),
            'precision_'+name : mean(scores['test_precision']),
            'recall_'+name : mean(scores['test_recall']),
            'roc_auc_'+name : mean(scores['test_roc_auc'])
        })
        print("Modelo: ", name)
        #print(scores)
#
wandb.agent(sweep_id2, function = train)

wandb: Agent Starting Run: 2v0u3eyv with config:
wandb: 	over_values: 0.3
wandb: 	under_values: 0.7
wandb: Currently logged in as: ignacio-onate. Use `wandb login --relogin` to force relogin


Modelo:  SVM
Modelo:  XGB
Modelo:  RF
Modelo:  LR


accuracy_LR,▁
accuracy_RF,▁
accuracy_SVM,▁
accuracy_XGB,▁
f1_LR,▁
f1_RF,▁
f1_SVM,▁
f1_XGB,▁
precision_LR,▁
precision_RF,▁
precision_SVM,▁


wandb: Agent Starting Run: 5kx86287 with config:
wandb: 	over_values: 0.3
wandb: 	under_values: 0.6


Modelo:  SVM
Modelo:  XGB
Modelo:  RF
Modelo:  LR


...

accuracy_LR,▁
accuracy_RF,▁
accuracy_SVM,▁
accuracy_XGB,▁
f1_LR,▁
f1_RF,▁
f1_SVM,▁
f1_XGB,▁
precision_LR,▁
precision_RF,▁
precision_SVM,▁


wandb: Agent Starting Run: js0vesuh with config:
wandb: 	over_values: 0.5
wandb: 	under_values: 0.5


Modelo:  SVM
Modelo:  XGB
Modelo:  RF
Modelo:  LR


accuracy_LR,▁
accuracy_RF,▁
accuracy_SVM,▁
accuracy_XGB,▁
f1_LR,▁
f1_RF,▁
f1_SVM,▁
f1_XGB,▁
precision_LR,▁
precision_RF,▁
precision_SVM,▁


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


# AJUSTE DE HIPERPARAMETROS
Tras los experimentos podemos concluir que el proceso que mejor balancea los datos es el de UP-OVER-SAMPLING con UP de 50% para la clase minoritaria y OVER de 50% para la clase mayoritaria.

Ahora fijamos este ratio para poner a prueba los modelos con sus respectivos hiperparametros.



In [ ]:
#Para esto debemos crear los diccionarios de hiperparametros.
